In [ ]:
from qdtuner import QuantumDotFET

# Connect to Tuner Class

In [ ]:
QD_FET_Tuner = QuantumDotFET(
    config='../config.yaml',
    tuner_config='../fet_tuner_config.yaml',
    station_config='../station_config.yaml',
    save_dir='C:\\Users\\coher\\Desktop\\Andrija'
)

Optional: Monitoring your variables

In [ ]:
# !python -m qcodes.monitor.monitor
# monitor = qc.Monitor(
#     QD_FET_Tuner.drain_volt, 
#     QD_FET_Tuner.voltage_source.S, 
#     QD_FET_Tuner.voltage_source.L,
#     QD_FET_Tuner.voltage_source.LB,
#     QD_FET_Tuner.voltage_source.P,
#     QD_FET_Tuner.voltage_source.RB,
#     )
# monitor.show()

In [ ]:
QD_FET_Tuner.ground_device()

# 0. Bias Device 

In [ ]:
QD_FET_Tuner.bias_ohmic(
    ohmic='S', 
    V=0.005
) 

In [ ]:
print(QD_FET_Tuner._get_drain_current())
QD_FET_Tuner.results

# 1. Turn On Device

In [ ]:
turn_on_data = QD_FET_Tuner.turn_on(
    minV=0, 
    maxV=None,
    dV=0.05
)

# 2. Pinch Off Gate(s)

In [ ]:
QD_FET_Tuner.deviceTurnsOn=True

QD_FET_Tuner.results['turn_on']['voltage'] = -2
QD_FET_Tuner.results['turn_on']['saturation'] = -3


In [ ]:
pinch_off_data = QD_FET_Tuner.pinch_off(
    minV=None, 
    maxV=None, 
    dV=0.05,
    voltage_configuration={}
)

In [ ]:
QD_FET_Tuner.results['LB']['pinch_off']['voltage'] = -2
QD_FET_Tuner.results['LB']['pinch_off']['width'] = 0.5
QD_FET_Tuner.results['RB']['pinch_off']['voltage'] = -2
QD_FET_Tuner.results['RB']['pinch_off']['width'] = 0.5

# 3. Barrier Barrier Sweep

In [ ]:
df, current_axes = QD_FET_Tuner.sweep_barriers(
    B1='LB', 
    B2='RB', 
    # B1_bounds=(None, None),
    # B2_bounds=(None, None),
    dV=0.05,
    voltage_configuration=None,
    extract_bias_point=False
)

Extract Bias Point 

In [ ]:
try: 
    bp, voltage_window = QD_FET_Tuner.extract_bias_point(
        df,
        plot_process=True,
        axes=current_axes
    )

except NameError:
    print("Most likely don't have detectron2 installed. Please enter bias point manually :\.")

In [ ]:
bias_point = {'LB': -1, 'RB': -1}

# Coulomb Blockade

In [ ]:
df, axes = QD_FET_Tuner.coulomb_blockade(
    gate='P', 
    gate_bounds=(0,0.75), 
    dV=0.05,
    voltage_configuration={'S': 0.005} | bias_point, 
)

Extract $\underset{\tilde{V}_P}{\mathrm{argmax}}\ G_{SD}$

In [ ]:
results = QD_FET_Tuner.extract_max_conductance_point(
    df,
    plot_process=True,
    sigma=0.1
)
for key, value in results.items():
    print(f"{key}: {value}")

# 5. Coulomb Diamonds

In [ ]:
df, cd_axes = QD_FET_Tuner.coulomb_diamonds(
    ohmic='S', 
    gate='P', 
    ohmic_bounds=(-0.015,0.015), #V
    gate_bounds=(0,0.75), #V
    dV_gate=0.03, 
    dV_ohmic=0.01,
    voltage_configuration=bias_point,
)

Extract lever arm, $\alpha$, at $\tilde{V}_{P}$

In [ ]:
results = QD_FET_Tuner.extract_lever_arms(
    df,
    plot_process=False
)

print(results)

In [ ]:
QD_FET_Tuner.voltage_source.print_readable_snapshot()

# Current Trace

In [ ]:
QD_FET_Tuner.current_trace(
    f_sampling=1000,
    t_capture=60, 
    plot_psd=True
)

# Ground Device

In [ ]:
QD_FET_Tuner.ground_device()

# DATABASE VIEWER

In [ ]:
db_file = "/Users/andrijapaurevic/Desktop/experiments_N44_D54_2024-05-08.db"
# import IPython.lib.backgroundjobs as bg
# from plottr.apps import inspectr

# jobs = bg.BackgroundJobManager()
# jobs.new(inspectr.main, db_file)

In [ ]:
db_file = "/Users/andrijapaurevic/Desktop/experiments_N44_D54_2024-05-08.db"


import qcodes as qc


import sys, os
import matplotlib.pyplot as plt 
qc.dataset.initialise_or_create_database_at(db_file)
tutorial_exp = qc.dataset.load_or_create_experiment(
    experiment_name="Initialization",
    sample_name="N44_D54"
)

dataset = qc.dataset.load_by_run_spec(experiment_name='Initialization', captured_run_id=16)
qc.dataset.plot_dataset(dataset)
df = dataset.to_pandas_dataframe().reset_index()


In [ ]:
import skimage
from skimage.transform import hough_line, hough_line_peaks, probabilistic_hough_line
from skimage.feature import canny
from skimage.draw import line as draw_line
from skimage import data
import matplotlib.cm as cm


range_RB = (0.615,0.747)
range_LB = (0.533,0.631)
filtered_df = df[
    (df['sim900_volt_RB'] >= range_RB[0]) & (df['sim900_volt_RB'] <= range_RB[1]) &
    (df['sim900_volt_LB'] >= range_LB[0]) & (df['sim900_volt_LB'] <= range_LB[1])
]

cropped_img, Xdata, Ydata = convert_data_to_image(filtered_df)
cropped_img = cropped_img[:,:,0]
edges = canny(cropped_img,sigma=0.5, low_threshold=0.1*np.iinfo(np.uint8).max, high_threshold=0.3 * np.iinfo(np.uint8).max)
lines = probabilistic_hough_line(edges, threshold=0, line_length=3,
                                 line_gap=0)
# Generating figure 2
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
ax = axes.ravel()

ax[0].imshow(cropped_img, cmap=cm.gray, origin='lower', extent=[Xdata.min(), Xdata.max(), Ydata.min() , Ydata.max()],)
ax[0].set_title('Input image')

ax[1].imshow(edges, cmap=cm.gray, origin='lower',  extent=[Xdata.min(), Xdata.max(), Ydata.min() , Ydata.max()],)
ax[1].set_title('Masked Canny edges')

potential_points = {}
angles_data = []
slopes_data = []
for line in lines:
    p0_pixel, p1_pixel = line
    p0, p1 = pixel_polygon_to_image_units(line, filtered_df)

    dy =  (p1[1]-p0[1])
    dx = (p1[0]-p0[0])
    if dx == 0:
        continue
    m = dy/dx
    theta = np.arctan(m)*(180/np.pi)
    if theta > -40 or theta < -60:
        continue
    angles_data.append(theta)
    slopes_data.append(m)
    midpoint_pixel = (np.array(p0_pixel) + np.array(p1_pixel))/2
    midpoint_units = (np.array(p0) + np.array(p1))/2
    # print(midpoint)
    midpoint = midpoint.astype(int)
    current_at_midpoint = filtered_df['agilent_volt'].to_numpy().reshape(len(Xdata), len(Ydata))[midpoint[0],midpoint[1]]
    potential_points[tuple(midpoint_units)] = current_at_midpoint

    ax[1].plot((p0[0], p1[0]), (p0[1], p1[1]))
    ax[1].scatter([midpoint_units[0]],[midpoint_units[1]], marker='*',s=50)
    ax[0].plot((p0[0], p1[0]), (p0[1], p1[1]))
    ax[0].scatter([midpoint_units[0]],[midpoint_units[1]], marker='*',s=50)
    
# ax[1].set_xlim((0, cropped_img.shape[1]))
# ax[1].set_ylim((cropped_img.shape[0], 0))
ax[1].set_title('Probabilistic Hough')
ax[2].hist(angles_data, bins=2*int(np.sqrt(len(slopes_data))))
ax[2].set_xlabel(r"$\theta^\circ$")
ax[2].set_ylabel(r"$f$")
# ax[2].hist(angles_data)
# ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
print(potential_points)
max_key = np.array(max(potential_points, key=potential_points.get))
print("Key with the highest value:", max_key)
bias_point = {Xdata.name: max_key[0], Ydata.name: max_key[1]}
I = filtered_df['agilent_volt'].to_numpy().reshape(len(Ydata), len(Xdata))
plt.imshow(I, extent=[Xdata.min(), Xdata.max(), Ydata.min() , Ydata.max()],origin='lower')
plt.scatter(*max_key, marker='*', c='r')
plt.ylabel(Ydata.name)
plt.xlabel(Xdata.name)
plt.colorbar()
plt.show()


print(f"Bias point: {bias_point}")